some ideas: https://stackoverflow.com/questions/41488279/neural-network-always-predicts-the-same-class
<pre>
change the split and downsampling order

now: start with an imbalanced dataset
     downsample all the data to a small balanced subset
     split and shape using stratify the small subset into 70/30 for Train/Test
     means the Test dataset is also balanced
     
want: start with imbalanced dataset
      split using sklearn:train_test_split into 70/30 for Train/Test
      downsample and shape the Train dataset to become balanced
      shape the Test dataset
      means the Test dataset is imbalanced as it should be


In [ ]:
import os
colab = os.environ.get('COLAB_GPU', '10')
if (int(colab) == 0):
    from google.colab import drive
    drive.mount('/content/drive')  
else:
    print("")

In [2]:
# Check if Google Colab path exists
if os.path.exists("/content/drive/My Drive/MyDSNotebooks/Imbalanced_data/input/creditcardzip") :
    # Change the current working Directory    
    os.chdir("/content/drive/My Drive/MyDSNotebooks/Imbalanced_data/input/creditcardzip")
# else check if Kaggle/local path exists
elif os.path.exists("../input/creditcardzip") :
    # Change the current working Directory    
    os.chdir("../input/creditcardzip")
else:
    print("Can't change the Current Working Directory") 
print("Current Working Directory " , os.getcwd())

import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import metrics 

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.recurrent import LSTM
from keras.layers import Embedding
#from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from keras import optimizers
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adagrad

a='''
# to try ---------------------------------------------
# snippet of using the LearningRateScheduler callback
from keras.callbacks import LearningRateScheduler
def my_learning_rate(epoch, lrate)
	return lrate 
lrs = LearningRateScheduler(my_learning_rate)
model.fit(..., callbacks=[lrs])

#or

# snippet of using the ReduceLROnPlateau callback
from keras.callbacks import ReduceLROnPlateau
...
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=100)
model.fit(..., callbacks=[rlrop])'''

# -------------------------------------------------------
#initialize variables

cm_results = []
class_names=[0,1] # name  of classes 1=fraudulent transaction

Current Working Directory  C:\DataScience\Repo\Imbalanced_data\input\creditcardzip


Using TensorFlow backend.


In [3]:
def display_metrics(model_name, train_features, test_features, train_label, test_label, pred):
    try:
        print(model_name.score(test_features, test_label)) 
        print("Accuracy score (training): {0:.3f}".format(model_name.score(train_features, train_label))) 
        print("Accuracy score (validation): {0:.3f}".format(model_name.score(test_features, test_label))) 
    except Exception as e:
        print("error")  
    try:
        print(pd.Series(model_name.feature_importances_, index=train_features.columns[:]).nlargest(10).plot(kind='barh')) 
    except Exception as e:
        print("error")  
        
    print("Confusion Matrix:")
    tn, fp, fn, tp = confusion_matrix(test_label, pred).ravel()
    total = tn+ fp+ fn+ tp 
    print("false positive pct:",(fp/total)*100) 
    print("tn", " fp", " fn", " tp") 
    print(tn, fp, fn, tp) 
    print(confusion_matrix(test_label, pred)) 
    print("Classification Report") 
    print(classification_report(test_label, pred))
    print("Specificity =", tn/(tn+fp))
    print("Sensitivity =", tp/(tp+fn))
    return tn, fp, fn, tp

In [4]:
df = pd.read_csv('creditcard.csv', na_filter=True)
X = df.loc[:, df.columns != 'Class']
y = df.loc[:, df.columns == 'Class']

#print(y['Class'].value_counts())
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = None, shuffle=True)

In [5]:
# prepare the Train and Test data for the model. Need input to include arrays with certain shape

# downsample the Train data

# find the number of minority (value=1) samples in our dataset so we can down-sample our majority to it
yes = len(y_train[y_train['Class'] ==1])

# retrieve the indices of the minority and majority samples 
yes_ind = y_train[y_train['Class'] == 1].index
no_ind = y_train[y_train['Class'] == 0].index

# random sample the majority indices based on the amount of 
# minority samples
new_no_ind = np.random.choice(no_ind, yes, replace = False)

# merge the two indices together
undersample_ind = np.concatenate([new_no_ind, yes_ind])

# get undersampled dataframe from the merged indices of the dataset
X_train = X_train.loc[undersample_ind]
y_temp = y_train.loc[undersample_ind]
y_train_original = np.array(y_temp['Class'], dtype='int')
y_test_original = np.array(y_test['Class'], dtype='int')

#show how the data was split between the 2 classes
print(y['Class'].value_counts())
unique_elements, counts_elements = np.unique(y_train_original, return_counts=True)
print("Frequency of unique values of the Train array:")
print(np.asarray((unique_elements, counts_elements)))
unique_elements, counts_elements = np.unique(y_test_original, return_counts=True)
print("Frequency of unique values of the Test array:")
print(np.asarray((unique_elements, counts_elements)))

#shape the Train data 
rolling_window_size = 10  ### this selects how many historical transactions should be analyzed to judge the transaction at hand -- RNN width

X_train_interim = np.zeros([(X_train.shape[0]-rolling_window_size)*rolling_window_size,30])
y_train = []
for i in range((X_train.shape[0]-rolling_window_size)):
    beg = 0+i
    end = beg+rolling_window_size
    s = np.array(X_train[beg:end], dtype='float')
    X_train_interim[(rolling_window_size*i):(rolling_window_size*(i+1)),:] = s
    y_train.append(y_train_original[end])

y_train = np.array(y_train, dtype='int')
X_train_interim = X_train_interim[:,1::]

X_train_tensor = X_train_interim.reshape(int(np.shape(X_train_interim)[0]/rolling_window_size), rolling_window_size, np.shape(X_train_interim)[1])
X_train = X_train_tensor

#shape and split the Test data 

#X_test_interim = np.zeros([(X_test.shape[0]-rolling_window_size)*10,30])
X_test_interim = np.zeros([(X_test.shape[0]-rolling_window_size)*rolling_window_size,30])
y_test = []
for i in range((X_test.shape[0]-rolling_window_size)):
    beg = 0+i
    end = beg+rolling_window_size
    s = np.array(X_test[beg:end], dtype='float')
    X_test_interim[(rolling_window_size*i):(rolling_window_size*(i+1)),:] = s
    y_test.append(y_test_original[end]) 

y_test = np.array(y_test, dtype='int')
X_test_interim = X_test_interim[:,1::]

X_test_tensor = X_test_interim.reshape(int(np.shape(X_test_interim)[0]/rolling_window_size), rolling_window_size, np.shape(X_test_interim)[1])
X_test = X_test_tensor

0    284315
1       492
Name: Class, dtype: int64
Frequency of unique values of the Train array:
[[  0   1]
 [366 366]]
Frequency of unique values of the Test array:
[[    0     1]
 [85317   126]]


In [7]:
# run model

### Hyperparameters Tuning
# First test optimal epochs holding everything else constant
# Dropout: 0.1-0.6
# GradientClipping: 0.1-10
# BatchSize: 32,64,128,256,512 (power of 2)

# setting variables
lr = 0.01
w1 = 4.0
w2 = 1.0
epochs = 20
batch_size = 8
min_ = 0
max_ = 2

for x in np.linspace(.0001, .01, num=2):
    for n in np.linspace(0, 3, num=2):
        ### Train LSTM using Keras 2 API ###
        model = Sequential()
        #model.add(LSTM(20, return_sequences=True, input_shape=X_train.shape[1:], kernel_initializer='lecun_uniform', activation='relu', kernel_regularizer=regularizers.l1(0.1), recurrent_regularizer=regularizers.l1(0.01), bias_regularizer=None, activity_regularizer=None, dropout=0.2, recurrent_dropout=0.2))#, return_sequences=True))
        #model.add(LSTM(20, return_sequences=True, input_shape=X_train.shape[1:], activation='relu', kernel_regularizer=regularizers.l1(0.1), recurrent_regularizer=regularizers.l1(0.01), bias_regularizer=None, activity_regularizer=None, dropout=0.2, recurrent_dropout=0.2))#, return_sequences=True))
        # defaults seem to work best, otherwise get very low score, AUC of 0.5 and all 0 or all 1 predictions
        model.add(LSTM(20, return_sequences=True, input_shape=X_train.shape[1:]))
        model.add(Dropout(0.2))
        model.add(LSTM(12, activation='relu', return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(8, activation='relu', return_sequences=False)) # this is the last LSTM, so should return_sequences=False
        #model.add(Dense(1, kernel_initializer='lecun_uniform', activation='sigmoid'))
        model.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

        #optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) # ValueError: None values not supported.
        optimizer = optimizers.Adam(lr=0.001)
        #model.compile(loss='mean_squared_error', optimizer='adam')
        #model.compile(loss='mean_squared_error', optimizer=adam)
        #model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy']) #optimizer='rmsprop', optimizer='sgd', optimizer='adam'

        a='''
        lr_schedule = optimizers.schedules.ExponentialDecay(
            initial_learning_rate=1e-2,
            decay_steps=10000,
            decay_rate=0.9)
        optimizer = optimizers.SGD(learning_rate=lr_schedule)'''

        #optimizer = optimizers.SGD(learning_rate=lr)
        #optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.9,decay=0.01)

        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #optimizer='rmsprop', optimizer='sgd', optimizer='adam'

        #print(model.summary())

        #model.fit(X_train, y_train, epochs=200, batch_size=10000, class_weight={0 : 1., 1: float(int(1/np.mean(y_train)))}, validation_split=0.3)
        #model.fit(X_train, y_train, epochs=4, batch_size=8)#, class_weight=np.where(y_train == 1,4.0,1.0).flatten() )
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, class_weight=np.where(y_train == 1,w1,w2).flatten() )

        train_predict = model.predict_classes(X_train)
        test_predict = model.predict_classes(X_test)

        ### test AUC ###

        fpr, tpr, thresholds = metrics.roc_curve(y_train, train_predict, pos_label=1)
        print('TRAIN | AUC Score: ' + str((metrics.auc(fpr, tpr))))
        fpr, tpr, thresholds = metrics.roc_curve(y_test, test_predict, pos_label=1)
        print('TEST | AUC Score: ' + str((metrics.auc(fpr, tpr))))
        tn, fp, fn, tp = display_metrics(model, X_train, X_test, y_train, y_test, test_predict)
        cm_results.append([model.name, tn, fp, fn, tp, lr, w1, w2, epochs, batch_size])
        print('lr',lr,'w1',w1,'w2',w2,'epochs',epochs,'batch_size',batch_size)
        del model

Epoch 1/20
722/722 [==============================] - 2s 3ms/step - loss: 0.4426 - accuracy: 0.8864
Epoch 2/20
722/722 [==============================] - 1s 1ms/step - loss: 0.0319 - accuracy: 0.9917
Epoch 3/20
722/722 [==============================] - 1s 1ms/step - loss: 0.0152 - accuracy: 0.9958
Epoch 4/20
722/722 [==============================] - 1s 2ms/step - loss: 0.0138 - accuracy: 0.9958
Epoch 5/20
722/722 [==============================] - 1s 2ms/step - loss: 0.0099 - accuracy: 0.9972
Epoch 6/20
722/722 [==============================] - 1s 1ms/step - loss: 0.0055 - accuracy: 0.9986
Epoch 7/20
722/722 [==============================] - 1s 1ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 8/20
722/722 [==============================] - 1s 2ms/step - loss: 6.0095e-04 - accuracy: 1.0000
Epoch 9/20
722/722 [==============================] - 1s 2ms/step - loss: 0.0066 - accuracy: 0.9972
Epoch 10/20
722/722 [==============================] - 1s 2ms/step - loss: 0.0011 - accuracy: 1.

In [8]:
final_results = pd.DataFrame(cm_results, columns=('algo','TN','FP','FN','TP', 'lr', 'w1', 'w2', 'epochs', 'batch_size')) 
#sp = round((tn1 + tn2)/(tn1 + tn2 +fp2), 3)
#se = round(tp2/(tp2 + fn1 + fn2), 3)
final_results['SP'] = round(final_results['TN']/(final_results['TN'] + final_results['FP']), 3)
final_results['SE'] = round(final_results['TP']/(final_results['TP'] + final_results['FN']), 3)
final_results['Avg'] = (final_results['SP'] + final_results['SE'])/2

filtered = final_results[~final_results.algo.str.contains('a', regex= True, na=False)]
sort = filtered.sort_values(filtered.columns[7], ascending = False)
print(sort)
sort.to_csv('results_lstm.csv', sep=',', mode='a', encoding='utf-8', header=True)

Empty DataFrame
Columns: [algo, TN, FP, FN, TP, lr, w1, w2, epochs, batch_size, SP, SE, Avg]
Index: []


In [ ]:
sort = final_results.sort_values(final_results.columns[5], ascending = False)
print(sort)